Solve problem when values disappear when the crack is not propagating

**Check popt = []** 

In [16]:
import os
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit

from ovito.io import *
from ovito.modifiers import *
from ovito.pipeline import *

# Packages form Henriks Github
from lammps_logfile import running_mean, get_color_value
from regex_file_collector import Collector
from regex_file_collector.utils import floating_number_pattern
import lammps_logfile

In [17]:
def sigmoid(x, c, b, a):
    z = b*(x-c)
    return a*np.where(z >= 0, 1 / (1 + np.exp(-z)), np.exp(z) / (1 + np.exp(z)))

def sigmoid(x, c, b, a):
    z = b*(x-c)
    return a*np.where(z >= 0, 1 / (1 + np.exp(-z)), np.exp(z) / (1 + np.exp(z)))

In [34]:
filename = '/home/users/marthgg/2021_05_simulations_quartz_relaxed_structure/angle_30/crack_simulation_deformZ_1.031881/trajectory.bin'

n_bins=10

reference_frame=7

    
# Read in trajectory-file
pipeline = import_file(filename, multiple_frames=True)

position_crack_tip = []
frames = []

frames_posCrack = []

extra_zeros = np.zeros((50))

for frame in range(reference_frame, pipeline.source.num_frames):

    # Coordination analysis:
    pipeline.modifiers.append(CoordinationAnalysisModifier(
        number_of_bins = n_bins, 
        partial = True))

    # Expression selection:
    pipeline.modifiers.append(ExpressionSelectionModifier(
        expression = 'Coordination>=4 || ParticleType==2 || Position.Y > 118 || Position.Y < 5 || Position.Z > 200 '))

    # Delete selected:
    pipeline.modifiers.append(DeleteSelectedModifier())

    # Spatial binning:
    pipeline.modifiers.append(SpatialBinningModifier(
        property = 'Coordination', 
        reduction_operation = SpatialBinningModifier.Operation.Sum, 
        direction = SpatialBinningModifier.Direction.X, 
        bin_count = (95, 200, 200)))

    #Extract and save data as a table
    data = pipeline.compute(frame)
    table = data.tables['binning'].xy()

    pos_x = table[:,0]
    coord = table[:,1]

    mean_pos_x = running_mean(pos_x, 10)
    mean_coord = running_mean(coord, 10)

    pos_x = mean_pos_x[7:-7]
    coord = mean_coord[7:-7]

    diff_pos = pos_x[1]-pos_x[0]

    # Add extra zeroes at the end to get a better fit
    new_pos = []
    for i in range (0, len(extra_zeros)):
        new_pos.append(pos_x[-1]+diff_pos*i)

    pos_x = np.insert(pos_x, len(pos_x), new_pos)
    coord = np.insert(coord, len(coord), extra_zeros)

    # Make a sigmoidal fit
    popt = [200]

    try:
        popt, pcov = curve_fit(sigmoid, pos_x, coord, p0=[popt[0], 0.001, (pos_x[0]+pos_x[-1])/2])
    except RuntimeError as e: 
        print(e)
        popt = [-1]

    #print(popt[0])

    position_crack_tip.append(popt[0])
    frames.append(frame)

    sig_x = np.linspace(pos_x[0], pos_x[-1], len(pos_x))
    sig_y = sigmoid(sig_x, *popt)

    plt.figure()
    plt.plot(pos_x, coord, label='data')
    plt.plot(sig_x, sig_y, label='fit')
    plt.legend()
    plt.show()

    position_crack_tip.append(popt[0])

Optimal parameters not found: gtol=0.000000 is too small, func(x) is orthogonal to the columns of
  the Jacobian to machine precision.


TypeError: sigmoid() missing 2 required positional arguments: 'b' and 'a'